In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard imports
import numpy as np
import xarray as xr
import tqdm as tqdm

In [ ]:
# Import OpenSense modules as submodules
import sys
import os

sys.path.append(os.path.abspath("./pycomlink/"))
sys.path.append(os.path.abspath("./poligrain/src/"))
sys.path.append(os.path.abspath("./mergeplg/src/"))

import pycomlink as pycml 
import poligrain as plg
import mergeplg 

# OpenMRG

In [ ]:
# OpenMRG
ds_rad = xr.open_dataset("data/andersson_2022_OpenMRG/radar/openmrg_rad.nc")                    
ds_cmls = xr.open_dataset("data/processed_cml_OpenMRG.nc").R_acc                                  

# Adjust rainfall fields

In [ ]:
os.makedirs('data/adjusted_fields', exist_ok=True)
months = ['2015-06', '2015-07', '2015-08']

In [ ]:
# IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_add_IDW']=xr.where(
    np.isnan(data['rainfall_add_IDW']), 0, data['rainfall_add_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_IDW.nc')    
del data, merger

In [ ]:
# POINT BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_pBK']=xr.where(
    np.isnan(data['rainfall_add_pBK']), 0, data['rainfall_add_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_pBK.nc')    
del data, merger

In [ ]:
# LINE BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_lBK']=xr.where(
    np.isnan(data['rainfall_add_lBK']), 0, data['rainfall_add_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenMRG_rainfall_add_lBK.nc')    
del data, merger

# OpenRainER

In [ ]:
# OpenRainER
ds_rad = xr.open_dataset("data/covi_2024_OpenRainER/openrainer_radar.nc")         
ds_cmls = xr.open_dataset("data/processed_cml_OpenRainER.nc").R_acc                     

# Adjust rainfall fields

In [ ]:
os.makedirs('data/adjusted_fields', exist_ok=True)
months = ['2022-07', '2022-08']

In [ ]:
# IDW
nnear = 12 
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_IDW": xr.concat(rainfall, dim="time") })
data['rainfall_add_IDW']=xr.where(
    np.isnan(data['rainfall_add_IDW']), 0, data['rainfall_add_IDW']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_IDW.nc')    
del data, merger

In [ ]:
# POINT BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_pBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_pBK']=xr.where(
    np.isnan(data['rainfall_add_pBK']), 0, data['rainfall_add_pBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_pBK.nc')    
del data, merger

In [ ]:
# LINE BLOCK KRIGING
variogram_parameters = {"sill": 0.9, "range": 10000, "nugget": 0.4}
nnear = 12
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"rainfall_add_lBK": xr.concat(rainfall, dim="time") })
data['rainfall_add_lBK']=xr.where(
    np.isnan(data['rainfall_add_lBK']), 0, data['rainfall_add_lBK']
    )
data.to_netcdf('data/adjusted_fields/OpenRainER_rainfall_add_lBK.nc')    
del data, merger